In [ ]:
#!unzip /content/drive/MyDrive/emotion_bw_face.zip -d /content/drive/MyDrive/fer2013

In [3]:
import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

import tensorflow as tf
import keras
from tensorflow.keras import layers
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau


#### Helper Functions

In [ ]:
def create_checkpoint(experiment_name, dir_name='drive/MyDrive/emotion_detection_project/checkpoints'):
    """
    Creates a chechkpoint file.

    Stores cp files with the filepath:
      "dir_name/experiment_name/current_datetime/"

    Args:
      dir_name: target directory to store cp files
      experiment_name: name of experiment directory (e.g. efficientnet_model_1)
    """
    checkpoint_path = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                    save_weights_only=True,
                                                    monitor='val_loss',
                                                    mode='min',
                                                    save_best_only=True)
    print(f"Saving checkpoint to (val_loss): {checkpoint_path}")
    return checkpoint

In [ ]:
def walk_through_dir(dir_path):
    """
    Walks through dir_path returning its contents.

    Args:
      dir_path (str): target directory

    Returns:
      A print out of:
        number of subdiretories in dir_path
        number of images (files) in each subdirectory
        name of each subdirectory
    """
    for dirpath, dirnames, filenames in os.walk(dir_path):
      print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# FER2013 Dataset

Dataset : [Facial Expression Recognition 2013](https://www.kaggle.com/datasets/msambare/fer2013)

The data consists of 48x48 pixel grayscale images of faces.The task is to categorize each face based on the emotion shown in the facial expression into one of seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral). The training set consists of 28,709 examples and the public test set consists of 3,589 examples.

In [1]:
train_dir = '/content/drive/MyDrive/fer2013/train'
test_dir = '/content/drive/MyDrive/fer2013/test'

In [ ]:
walk_through_dir('/content/drive/MyDrive/fer2013')

There are 2 directories and 0 images in '/content/drive/MyDrive/fer2013'.
There are 7 directories and 0 images in '/content/drive/MyDrive/fer2013/test'.
There are 0 directories and 958 images in '/content/drive/MyDrive/fer2013/test/angry'.
There are 0 directories and 111 images in '/content/drive/MyDrive/fer2013/test/disgusted'.
There are 0 directories and 1024 images in '/content/drive/MyDrive/fer2013/test/fearful'.
There are 0 directories and 1774 images in '/content/drive/MyDrive/fer2013/test/happy'.
There are 0 directories and 1233 images in '/content/drive/MyDrive/fer2013/test/neutral'.
There are 0 directories and 1247 images in '/content/drive/MyDrive/fer2013/test/sad'.
There are 0 directories and 831 images in '/content/drive/MyDrive/fer2013/test/surprised'.
There are 7 directories and 0 images in '/content/drive/MyDrive/fer2013/train'.
There are 0 directories and 3995 images in '/content/drive/MyDrive/fer2013/train/angry'.
There are 0 directories and 436 images in '/content/dri

## Get Data Ready For Training

In [31]:
IMG_SIZE = (48,48)
BATCH_SIZE = 32

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=IMG_SIZE,
                                                                 color_mode='rgb',
                                                                 label_mode='categorical',
                                                                 batch_size=BATCH_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                 image_size=IMG_SIZE,
                                                                 color_mode='rgb',
                                                                 label_mode='categorical',
                                                                 batch_size=BATCH_SIZE)

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [18]:
train_data

<_BatchDataset element_spec=(TensorSpec(shape=(None, 48, 48, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>

In [19]:
train_data.class_names

['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

In [ ]:
# Take of one batch
for images,labels in train_data.take(1):
    print(images, labels)

# Models

##Model0 EfficientNetB7
* Preprocess-> None
* Model-> Feature Extractor: EfficientNetB0(pretrained on ImageNet, all layers frozen) with no top.

In [27]:
model = EfficientNetB7(weights='imagenet')
base_model = tf.keras.applications.EfficientNetB7(include_top=False)
base_model.trainable = False
inputs = tf.keras.layers.Input(shape=(48,48,3), name='input_layer')
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pool_layer')(x)
outputs = tf.keras.layers.Dense(7, activation='softmax', name='output_layer')(x)
model_0 = tf.keras.Model(inputs, outputs)
model_0.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 48, 48, 3)]       0         
                                                                 
 efficientnetb7 (Functional)  (None, None, None, 2560)  64097687 
                                                                 
 global_avg_pool_layer (Glob  (None, 2560)             0         
 alAveragePooling2D)                                             
                                                                 
 output_layer (Dense)        (None, 7)                 17927     
                                                                 
Total params: 64,115,614
Trainable params: 17,927
Non-trainable params: 64,097,687
_________________________________________________________________


In [28]:
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                metrics=['accuracy'])

In [29]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.001)

In [ ]:
history_model_0 = model_0.fit(train_data,
                              epochs=25,
                              steps_per_epoch=len(train_data),
                              validation_data=test_data,
                              validation_steps=int(0.5*len(test_data)),
                              callbacks=[reduce_lr])

In [ ]:
model_0.evaluate(test_data)

113/113 [==============================] - 78s 689ms/step - loss: 1.1602 - accuracy: 0.5711


[1.160204529762268, 0.5710504055023193]

##Model1 Data Augmentation Layer
* Preprocessing-> Data Augmentation with Original image size
* Model-> MobileNetV2

In [ ]:
IMG_SIZE = (48,48)
BATCH_SIZE = 64

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=IMG_SIZE,
                                                                 color_mode='rgb',
                                                                 label_mode='categorical',
                                                                 batch_size=BATCH_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                 image_size=IMG_SIZE,
                                                                 color_mode='rgb',
                                                                 label_mode='categorical',
                                                                 batch_size=BATCH_SIZE)

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [ ]:
data_aug = keras.Sequential([
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2)
], name='data_augmentation')

In [ ]:
base_model = tf.keras.applications.MobileNetV2(include_top=False)
base_model.trainable = False
inputs = tf.keras.layers.Input(shape=(48,48,3), name='input_layer')
x = data_aug(inputs)
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pool_layer')(x)
outputs = tf.keras.layers.Dense(7, activation='softmax', name='output_layer')(x)
model_1 = tf.keras.Model(inputs, outputs)

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
model_1.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
history_model_1 = model_1.fit(train_data,
                              epochs=20,
                              steps_per_epoch=len(train_data),
                              validation_data=test_data,
                              validation_steps=int(0.5*len(test_data)),
                              callbacks=[callback,create_tensorboard_callback('model_1')])

Saving TensorBoard log files to: drive/MyDrive/emotion_detection_project/logs/model_1/20230811-010232
Epoch 1/20
449/449 [==============================] - 24s 52ms/step - loss: 1.8101 - accuracy: 0.2525 - val_loss: 1.8103 - val_accuracy: 0.2455 - lr: 0.0010
Epoch 2/20
449/449 [==============================] - 23s 50ms/step - loss: 1.8087 - accuracy: 0.2522 - val_loss: 1.8102 - val_accuracy: 0.2425 - lr: 0.0010
Epoch 3/20
449/449 [==============================] - 23s 50ms/step - loss: 1.8079 - accuracy: 0.2521 - val_loss: 1.8103 - val_accuracy: 0.2436 - lr: 0.0010
Epoch 4/20
449/449 [==============================] - 23s 50ms/step - loss: 1.8074 - accuracy: 0.2521 - val_loss: 1.8117 - val_accuracy: 0.2427 - lr: 0.0010
Epoch 5/20
449/449 [==============================] - 25s 55ms/step - loss: 1.8071 - accuracy: 0.2521 - val_loss: 1.8096 - val_accuracy: 0.2430 - lr: 0.0010
Epoch 6/20
449/449 [==============================] - 25s 55ms/step - loss: 1.8070 - accuracy: 0.2522 - val_loss:

KeyboardInterrupt: ignored

In [ ]:
model_1.evaluate(test_data)

In [ ]:
model_path = 'drive/MyDrive/emotion_detection_project/models/model_1.h5'
model_1.save(model_path)

##Model2 ResNEt50V2


In [11]:
IMG_SIZE = (48,48)
BATCH_SIZE = 64

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                 image_size=IMG_SIZE,
                                                                 color_mode='rgb',
                                                                 label_mode='categorical',
                                                                 batch_size=BATCH_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=IMG_SIZE,
                                                                 color_mode='rgb',
                                                                 label_mode='categorical',
                                                                 batch_size=BATCH_SIZE)

Found 7178 files belonging to 7 classes.
Found 28709 files belonging to 7 classes.


In [12]:
base_model = tf.keras.applications.ResNet50V2(include_top=False)
base_model.trainable = False
inputs = tf.keras.layers.Input(shape=(48,48,3), name='input_layer')
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pool_layer')(x)
outputs = tf.keras.layers.Dense(7, activation='softmax', name='output_layer')(x)
model_2 = tf.keras.Model(inputs, outputs)

In [6]:
model_2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 48, 48, 3)]       0         
                                                                 
 resnet50v2 (Functional)     (None, None, None, 2048)  23564800  
                                                                 
 global_avg_pool_layer (Glob  (None, 2048)             0         
 alAveragePooling2D)                                             
                                                                 
 output_layer (Dense)        (None, 7)                 14343     
                                                                 
Total params: 23,579,143
Trainable params: 14,343
Non-trainable params: 23,564,800
_________________________________________________________________


In [15]:
model_2.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [13]:
def scheduler(epoch, lr):
  if epoch % 5 == 0 and epoch > 0:
     return lr * tf.math.exp(-0.1)
  else:
     return lr
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [16]:
history_model_2 = model_2.fit(train_data,
                              epochs=20,
                              steps_per_epoch=len(train_data),
                              validation_data=test_data,
                              validation_steps=int(0.001*len(test_data)),
                              callbacks=[callback])

Epoch 1/20
 32/113 [=======>......................] - ETA: 15:57 - loss: 66.8032 - accuracy: 0.1704

KeyboardInterrupt: ignored